<a href="https://colab.research.google.com/github/heber-augusto/udacity-intro-to-ml/blob/master/svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This jupyter notebook was created during Intro to Machine Learning Udacity course.

The goal at this notebook is to train and predict using SVM classifier.

This notebook demonstrate:

 - Getting the dataset and putting its contents into google drive;
 - Trainning using scikit learn svm classifier;
 - Print some timing and accuracy results


The next cells mount google drive to colab notebook

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Changes the folder and git clone a repo (which was not used in this notebook).

In [2]:
%cd /content/drive/My\ Drive/repos
!git clone https://github.com/heber-augusto/udacity-intro-to-ml
%cd /content/drive/My\ Drive/repos/udacity-intro-to-ml

/content/drive/My Drive/repos/udacity-intro-to-ml


Get the file containg datasets

In [3]:
!wget -P ../udacity-intro-to-ml-files http://www.cs.cmu.edu/~./enron/enron_mail_20150507.tar.gz

Extract all files

In [4]:
print ("unzipping Enron dataset (this may take a while)")
import tarfile
import os

tfile = tarfile.open("../udacity-intro-to-ml-files/enron_mail_20150507.tar.gz", "r:gz")
tfile.extractall("../udacity-intro-to-ml-files/.")

print ("you're ready to go!")

unzipping Enron dataset (this may take a while)
you're ready to go!


The "preprocess" function is the same which is found inside github repository. The original version (from Udacity) was changed to a python3 and google colab compatible version.

In [5]:
import pickle
import numpy

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif



def preprocess(
    words_file = "../tools/word_data.pkl", 
    authors_file="../tools/email_authors.pkl"):
    """ 
        this function takes a pre-made list of email texts (by default word_data.pkl)
        and the corresponding authors (by default email_authors.pkl) and performs
        a number of preprocessing steps:
            -- splits into training/testing sets (10% testing)
            -- vectorizes into tfidf matrix
            -- selects/keeps most helpful features
        after this, the feaures and labels are put into numpy arrays, which play nice with sklearn functions
        4 objects are returned:
            -- training/testing features
            -- training/testing labels
    """

    ### the words (features) and authors (labels), already largely preprocessed
    ### this preprocessing will be repeated in the text learning mini-project
    authors_file_handler = open(authors_file, "rb")
    authors = pickle.load(authors_file_handler)
    authors_file_handler.close()

    words_file_handler = open(words_file, "rb")
    word_data = pickle.load(words_file_handler)
    words_file_handler.close()

    ### test_size is the percentage of events assigned to the test set
    ### (remainder go into training)
    features_train, features_test, labels_train, labels_test = train_test_split(
        word_data, 
        authors, 
        test_size=0.1, 
        random_state=42)



    ### text vectorization--go from strings to lists of numbers
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
    features_train_transformed = vectorizer.fit_transform(features_train)
    features_test_transformed  = vectorizer.transform(features_test)



    ### feature selection, because text is super high dimensional and 
    ### can be really computationally chewy as a result
    selector = SelectPercentile(f_classif, percentile=10)
    selector.fit(features_train_transformed, labels_train)
    features_train_transformed = selector.transform(features_train_transformed).toarray()
    features_test_transformed  = selector.transform(features_test_transformed).toarray()

    ### info on the data
    print (f"no. of Chris training emails: {sum(labels_train)}")
    print (f"no. of Sara training emails: {len(labels_train)-sum(labels_train)}")
    
    return features_train_transformed, features_test_transformed, labels_train, labels_test

In [6]:
features_train, features_test, labels_train, labels_test = preprocess(
    words_file = "./tools/word_data.pkl", 
    authors_file="./tools/email_authors.pkl")

no. of Chris training emails: 7936
no. of Sara training emails: 7884


**Traing a SVM model with linear kernel**

In [12]:
from sklearn import svm
from time import time
clf = svm.LinearSVC()

t0 = time()
model_trained = clf.fit(features_train, labels_train)
print (f"training time: {round(time()-t0, 3)} s")

t1 = time()
labels_pred = model_trained.predict(features_test)
print (f"prediction time: {round(time()-t1, 3)} s")
print("Number of mislabeled points out of a total %d points : %d"
    % (features_test.shape[0], (labels_test != labels_pred).sum()))

training time: 0.38 s
prediction time: 0.01 s
Number of mislabeled points out of a total 1758 points : 18


In [13]:
accuracy = (features_test.shape[0] - (labels_test != labels_pred).sum()) / features_test.shape[0]
print (f"accuracy: {accuracy}")

accuracy: 0.9897610921501706


In [41]:
features_train.shape

(15820, 3785)

**Reducing training dataset to 1% to compare the timming and accuracy**

In [28]:
features_train_small = features_train[:int(features_train.shape[0]/100)]
labels_train_small = labels_train[:int(len(labels_train)/100)]

clf_small_training_set = svm.LinearSVC()

t0 = time()
model_trained_small = clf_small_training_set.fit(
    features_train_small, 
    labels_train_small)
print (f"training time: {round(time()-t0, 3)} s")

t1 = time()
labels_pred = model_trained_small.predict(features_test)
print (f"prediction time: {round(time()-t1, 3)} s")
print("Number of mislabeled points out of a total %d points : %d"
    % (features_test.shape[0], (labels_test != labels_pred).sum()))

accuracy = (features_test.shape[0] - (labels_test != labels_pred).sum()) / features_test.shape[0]
print (f"accuracy: {accuracy}")


training time: 0.007 s
prediction time: 0.01 s
Number of mislabeled points out of a total 1758 points : 179
accuracy: 0.8981797497155859


**Train a SVM model with rbf kernel and just 1% of the training set**

In [30]:
clf_rbf = svm.SVC(kernel='rbf')

t0 = time()
model_trained_rbf = clf_rbf.fit(
    features_train_small, 
    labels_train_small)
print (f"training time: {round(time()-t0, 3)} s")

t1 = time()
labels_pred = model_trained_rbf.predict(features_test)
print (f"prediction time: {round(time()-t1, 3)} s")
print("Number of mislabeled points out of a total %d points : %d"
    % (features_test.shape[0], (labels_test != labels_pred).sum()))

accuracy = (features_test.shape[0] - (labels_test != labels_pred).sum()) / features_test.shape[0]
print (f"accuracy: {accuracy}")

training time: 0.155 s
prediction time: 1.574 s
Number of mislabeled points out of a total 1758 points : 184
accuracy: 0.8953356086461889


**Grid search with different C values to check accuracy**

In [42]:
from sklearn.model_selection import GridSearchCV

C_range = [10, 100, 1000., 10000.]
parameters = {'kernel':('rbf',), 'C':C_range}
svc = svm.SVC()
grid = GridSearchCV(svc, parameters)
grid.fit(
    features_train_small, 
    labels_train_small)

print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 10, 'kernel': 'rbf'} with a score of 0.87
